In [1]:
import pandas as pd
import numpy as np

In [2]:
traffic_df = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/cleaned/traffic_table_cleaned.csv')
route_df = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/cleaned/routes_table.csv')
city_weather_df = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/cleaned/city_weather_cleaned.csv')
drivers_table_df = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/cleaned/drivers_table.csv')
routes_table = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/raw/routes_table.csv')
route_weather_df = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/cleaned/routes_weather_cleaned.csv')
truck_schedule_df = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/cleaned/truck_schedule_table_cleaned.csv')
truck_df = pd.read_csv('/Users/jeevanapathipati/Documents/TruckDelay/data/cleaned/trucks_table_cleaned.csv')

In [3]:
# 1. Merge driver_df with truck_df on vehicle_no and truck_id
merged_driver_truck_df = pd.merge(truck_df, drivers_table_df, left_on='truck_id', right_on='vehicle_no', how='inner')
#merged_driver_truck_df=merged_driver_truck_df.drop('vehicle_no',axis=1)


In [4]:
merged_driver_truck_df=merged_driver_truck_df.drop('vehicle_no',axis=1)

In [5]:
merged_driver_truck_df.shape

(1086, 13)

In [6]:
# 2. Merge merged_driver_truck_df with truck_schedule_df on truck_id
merged_schedule_df = pd.merge(merged_driver_truck_df, truck_schedule_df, on='truck_id', how='inner')



In [7]:
merged_schedule_df.shape

(10197, 17)

In [8]:
merged_schedule_df.columns

Index(['truck_id', 'truck_age', 'load_capacity_pounds', 'mileage_mpg',
       'fuel_type', 'driver_id', 'name', 'gender', 'age', 'experience',
       'driving_style', 'ratings', 'average_speed_mph', 'route_id',
       'departure_date', 'estimated_arrival', 'delay'],
      dtype='object')

In [9]:
# 3. Extract date and hour from the departure_date in truck_schedule_df for merging with traffic_df
merged_schedule_df['date'] = pd.to_datetime(merged_schedule_df['departure_date']).dt.date


In [10]:
merged_schedule_df.columns

Index(['truck_id', 'truck_age', 'load_capacity_pounds', 'mileage_mpg',
       'fuel_type', 'driver_id', 'name', 'gender', 'age', 'experience',
       'driving_style', 'ratings', 'average_speed_mph', 'route_id',
       'departure_date', 'estimated_arrival', 'delay', 'date'],
      dtype='object')

In [11]:
# merged_schedule_df['hour'] = pd.to_datetime(merged_schedule_df['departure_date']).dt.hour
# 4. Merge merged_schedule_df with traffic_df on route_id, date, and hour

# 5. Merge merged_traffic_df with route_df on route_id
merged_route_df = pd.merge(merged_schedule_df, routes_table, on='route_id', how='inner')




In [12]:
traffic_df['date']=pd.to_datetime(traffic_df['date']).dt.date


In [13]:
merged_route_df.columns

Index(['truck_id', 'truck_age', 'load_capacity_pounds', 'mileage_mpg',
       'fuel_type', 'driver_id', 'name', 'gender', 'age', 'experience',
       'driving_style', 'ratings', 'average_speed_mph', 'route_id',
       'departure_date', 'estimated_arrival', 'delay', 'date', 'origin_id',
       'destination_id', 'distance', 'average_hours'],
      dtype='object')

In [14]:
merged_traffic_df = pd.merge(merged_route_df, traffic_df, on=['route_id', 'date'], how='inner')

In [16]:
merged_traffic_df.shape

(244853, 25)

In [18]:
route_weather_df['Date']=pd.to_datetime(route_weather_df['Date']).dt.date
merged_route_weather_df = pd.merge(merged_route_df, route_weather_df, left_on=['route_id', 'date'], right_on=['route_id', 'Date'], how='inner')

In [19]:
merged_route_weather_df.shape

(37982, 30)

In [20]:
merged_route_weather_df.columns

Index(['truck_id', 'truck_age', 'load_capacity_pounds', 'mileage_mpg',
       'fuel_type', 'driver_id', 'name', 'gender', 'age', 'experience',
       'driving_style', 'ratings', 'average_speed_mph', 'route_id',
       'departure_date', 'estimated_arrival', 'delay', 'date', 'origin_id',
       'destination_id', 'distance', 'average_hours', 'Date', 'temp',
       'wind_speed', 'description', 'precip', 'humidity', 'visibility',
       'pressure'],
      dtype='object')

In [ ]:
# Univariate Analysis: Plotting distributions of individual variables
def plot_univariate(df, column, plot_type='hist'):
    plt.figure(figsize=(10, 6))
    if plot_type == 'hist':
        sns.histplot(df[column], kde=True, color='blue')
    elif plot_type == 'box':
        sns.boxplot(x=df[column], color='green')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()
    
    
# Bivariate Analysis: Exploring relationships between two variables
def plot_bivariate(df, x_column, y_column, plot_type='scatter'):
    plt.figure(figsize=(10, 6))
    if plot_type == 'scatter':
        sns.scatterplot(data=df, x=x_column, y=y_column)
    elif plot_type == 'line':
        sns.lineplot(data=df, x=x_column, y=y_column)
    plt.title(f'Relationship between {x_column} and {y_column}')
    plt.xlabel(x_column)
    plt.ylabel(y_column)
    plt.grid(True)
    plt.show()
    
# Multivariate Analysis: Exploring complex interactions between multiple variables
def plot_multivariate(df, plot_type='pair', columns=None):
    plt.figure(figsize=(12, 10))
    if plot_type == 'pair':
        sns.pairplot(df[columns])
    plt.show()


# Example Univariate Analysis
# plot_univariate(city_weather_df, 'temp')  # Histogram of temperatures
# plot_univariate(city_weather_df, 'wind_speed')
# plot_univariate(city_weather_df, 'humidity')
# plot_univariate(city_weather_df, 'pressure')
# plot_univariate(city_weather_df, 'description', 'box')  # Boxplot of driver ages
# Example Bivariate Analysis
# plot_bivariate(city_weather_df, 'temp', 'humidity',plot_type='line')  # Scatter plot of temperature vs. humidity

# Example Multivariate Analysis
# plot_multivariate(city_weather_df, 'pair', columns=['temp', 'wind_speed', 'humidity'])  # Pairplot of weather variables